In [ ]:
title: "PCAadapt_v1"
author: "Caroline Rzucidlo"
date: "November 18, 2021"
output: html_document
---
# useful websites: https://cran.r-project.org/web/packages/pcadapt/vignettes/pcadapt.html, https://popgen.nescent.org/2016-01-26-SNP-selection.html, https://www.rdocumentation.org/packages/pcadapt/versions/3.0.4
install.packages("devtools")
install.packages("pcadapt")
install.packages("dplyr")
install.packages("BiocManager")
BiocManager::install("qvalue")
install.packages("cowplot")
if (!("OutFLANK" %in% installed.packages())){install_github("whitlock/OutFLANK")}
if (!("vcfR" %in% installed.packages())){install.packages("vcfR")}

library("devtools")
#source("https://bioconductor.org/biocLite.R")
library("pcadapt")
library("qvalue")
library("OutFLANK")
library("ggplot2")
library("dplyr")
library("cowplot")
library("vcfR")

# set the working directory

# import the vcf file
path <- "vortexfs1/omics/env-bio/collaboration/dino_popgen/output/snp_calling/allhqSNPmm80.recode.vcf"
vcf <- read.pcadapt(path, type = "vcf", type.out = "matrix", ploidy = 1)

# chose the number of K principal components
pc <- pcadapt(input = vcf, K = 10)
summary(pc)

# make a scree plot
plot(pc, option = "screeplot")

# make a manhattan plot

#K <- 2 # Index of the principal component the user is interested in
m <- plot(pc, option = "manhattan", threshold = 0.5)
save_plot("mm80_manhattan.png", m, base_aspect_ratio = 1.4)

# make a Q-Q plot
plot(pc, option = "qqplot", threshold = 0.05)

# plot stats distribution 
# Distribution of Mahalanobis distances
plot(pc, option = "stat.distribution", threshold = 0.05)

# determine the outliers
qval <- qvalue(pc$pvalues)$qvalues
alpha <- 0.05
outliers_pcadapt <- which(qval < alpha)
length(outliers_pcadapt)
#print(outliers_pcadapt)

# make a data frame of pvalues
pvalz <- as.data.frame(pc$pvalues) 
pvalz$ID <- seq.int(nrow(pvalz))
colnames(pvalz) <- c("PCA p-val", "ID")

# make a data frame of pvalues
qvalz <- as.data.frame(qval)
qvalz$ID <- seq.int(nrow(qvalz))
colnames(qvalz) <- c("PCA q-val", "ID")

# merge tha q-value and p-value dataframes
valz <- merge(pvalz, qvalz, by="ID")
# write out the data for ALL SNPs - not just outliers
write.table(valz, "../output/seq_outliers/PCADAPT_ALLvalues.txt", sep="\t")

# make a data frame for the outliers only
out <- as.data.frame(outliers_pcadapt)
colnames(out) <- ("ID")
# use a semi_join() so it only includes outlier values
outliers <- semi_join(valz, out, by = "ID", copy = FALSE)

# write out the data for the OUTLIERS ONLY
write.table(outliers, "../output/seq_outliers/outliersONLY_PCADAPT.txt", sep="\t")

# find outlier overlap with bayescan data
# > merge(a,b, by.x = "c1", by.y = "a")
all <- read.table("../output/seq_outliers/bayescan_outliers_clean.txt")
overlap <- merge(all, outliers, by = "ID")

### save the overlap table!!!
write.table(overlap, "../output/seq_outliers/snp_under_sel_methods_OVERLAP.txt", sep = "\t")


```
